<a href="https://colab.research.google.com/github/ElnazDi/colab/blob/master/Panda_Grouping_Sorting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import files
file = files.upload()

Saving winemag-data-130k-v2.csv to winemag-data-130k-v2.csv


In [11]:
reviews=pd.read_csv('winemag-data-130k-v2.csv', index_col=0)
pd.set_option("display.max_rows", 5)
reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [12]:
reviews.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title',
       'variety', 'winery'],
      dtype='object')

In [24]:
reviews.points

0         87
1         87
          ..
129969    90
129970    90
Name: points, Length: 129971, dtype: int64

In [17]:
reviews.points.unique()

array([ 87,  86,  85,  88,  92,  91,  90,  89,  83,  82,  81,  80, 100,
        98,  97,  96,  95,  93,  94,  84,  99])

In [20]:
reviews.points.value_counts()  # Count uniques

88     17207
87     16933
       ...  
99        33
100       19
Name: points, Length: 21, dtype: int64


# Grouping

**groupby** created groups of reviews with the same point values. 

Then, for **each group**, it **counted the size of the group**.



In [21]:

reviews.groupby('points').points.count()  # Count uniques like before

points
80     397
81     692
      ... 
99      33
100     19
Name: points, Length: 21, dtype: int64

In [25]:
reviews.groupby('points').price.min()

points
80      5.0
81      5.0
       ... 
99     44.0
100    80.0
Name: price, Length: 21, dtype: float64

You can think of **each group** we generate as being a **slice of our DataFrame** containing only data with values that match.

This DataFrame is accessible to us directly using the **apply** method, and we can then manipulate the data in any way we see fit. 

For example, here's one way of selecting the name of the first wine reviewed from each winery in the dataset:



In [27]:
reviews.groupby('winery').apply(lambda DataFrame:DataFrame.title.iloc[0])  

#Not sure but I think contact winery unique to title that its name come in first row

#notice is only on winery uniques = 16757

winery
1+1=3                          1+1=3 NV Rosé Sparkling (Cava)
10 Knots                 10 Knots 2010 Viognier (Paso Robles)
                                  ...                        
àMaurice    àMaurice 2013 Fred Estate Syrah (Walla Walla V...
Štoka                         Štoka 2009 Izbrani Teran (Kras)
Length: 16757, dtype: object

In [31]:
reviews.winery.describe()

count                 129971
unique                 16757
top       Wines & Winemakers
freq                     222
Name: winery, dtype: object

In [32]:
reviews.winery.unique()

array(['Nicosia', 'Quinta dos Avidagos', 'Rainstorm', ...,
       'Mas de Pampelonne', 'Bodegas Eidosela', 'Penedo Borges'],
      dtype=object)

In [33]:
reviews.title.unique()

array(['Nicosia 2013 Vulkà Bianco  (Etna)',
       'Quinta dos Avidagos 2011 Avidagos Red (Douro)',
       'Rainstorm 2013 Pinot Gris (Willamette Valley)', ...,
       'Domaine Gresser 2013 Kritt Gewurztraminer (Alsace)',
       'Domaine Marcel Deiss 2012 Pinot Gris (Alsace)',
       'Domaine Schoffit 2012 Lieu-dit Harth Cuvée Caroline Gewurztraminer (Alsace)'],
      dtype=object)

In [34]:
reviews.title.describe()

count                                                129971
unique                                               118840
top       Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma...
freq                                                     11
Name: title, dtype: object

In [28]:
reviews.shape

(129971, 13)

For even more fine-grained control, you can also **group by more than one column**. For an example, here's how we would pick out the best wine by country and province:



[argmax()](https://scaron.info/blog/argmax-in-python.html)

In [35]:
reviews.groupby(['country', 'province']).apply(lambda df: df.loc[df.points.argmax()])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  """Entry point for launching an IPython kernel.


country  ...                winery
country   province                     ...                      
Argentina Mendoza Province  Argentina  ...  Bodega Catena Zapata
          Other             Argentina  ...                Colomé
...                               ...  ...                   ...
Uruguay   San Jose            Uruguay  ...        Castillo Viejo
          Uruguay             Uruguay  ...               Narbona

[425 rows x 13 columns]

Another groupby method worth mentioning is agg, which lets you run a bunch of different functions on your DataFrame simultaneously. For example, we can generate a simple statistical summary of the dataset as follows:



In [38]:
reviews.groupby('country').price.agg([len,min,max])

,len,min,max
country,,,
Argentina,3800.0,4.0,230.0
Armenia,2.0,14.0,15.0
...,...,...,...
Ukraine,14.0,6.0,13.0
Uruguay,109.0,10.0,130.0


# Multi-indexes

The examples you've seen so far worked with DataFrame or Series objects with a **single-label index**. 

**groupby is slightly different because it will sometimes result in a "multi-index.**"

A multi-index has multiple levels. For example:

In [41]:
countries_reviewed= reviews.groupby(['country','province']).description.agg([len])
countries_reviewed

len
country   province              
Argentina Mendoza Province  3264
          Other              536
...                          ...
Uruguay   San Jose             3
          Uruguay             24

[425 rows x 1 columns]

Multi-indices have several methods for dealing with their tiered structure which are absent for single-level indices.

They also require two levels of labels to retrieve a value, an operation that looks something like this. Dealing with multi-index output is a common "gotcha" for users new to pandas.

The use cases for a MultiIndex are detailed alongside detailed instructions on using them in the[ MultiIndex / Advanced Selection section of the pandas documentation](https://pandas.pydata.org/pandas-docs/stable/advanced.html).

But the MultiIndex method you will use most often is the one for converting back to a regular index, the reset_index method:

In [42]:
countries_reviewed.reset_index()


,country,province,len
0,Argentina,Mendoza Province,3264
1,Argentina,Other,536
...,...,...,...
423,Uruguay,San Jose,3
424,Uruguay,Uruguay,24


# Sorting

Grouping returns data in index order, not in value order. That is to say, when outputting the result of a groupby, the order of the rows is dependent on the values in the index, not the data.

But you can sort the data with the **sort_values** method.

In [43]:
countries_reviewed = countries_reviewed.reset_index()
countries_reviewed.sort_values(by='len')

,country,province,len
179,Greece,Muscat of Kefallonian,1
192,Greece,Sterea Ellada,1
...,...,...,...
415,US,Washington,8639
392,US,California,36247


**sort_values** defaults to an **ascending sort**, where the lowest values go first.

Most of the time we want a **descending** sort however, where the higher numbers go first. That goes thusly:



In [44]:
countries_reviewed.sort_values(by='len', ascending=False)


,country,province,len
392,US,California,36247
415,US,Washington,8639
...,...,...,...
63,Chile,Coelemu,1
149,Greece,Beotia,1


To sort by index values, use the companion method **sort_index.** This method has the same arguments and default order:



In [45]:
countries_reviewed.sort_index()


,country,province,len
0,Argentina,Mendoza Province,3264
1,Argentina,Other,536
...,...,...,...
423,Uruguay,San Jose,3
424,Uruguay,Uruguay,24


Finally, know that you can sort by more than one column at a time:



In [46]:
countries_reviewed.sort_values(by=['country', 'len'])


,country,province,len
1,Argentina,Other,536
0,Argentina,Mendoza Province,3264
...,...,...,...
424,Uruguay,Uruguay,24
419,Uruguay,Canelones,43


## 1.
Who are the most common wine reviewers in the dataset? Create a `Series` whose index is the `taster_twitter_handle` category from the dataset, and whose values count how many reviews each person wrote.

In [48]:
reviews_written = reviews.groupby('taster_twitter_handle').size()
reviews_written

#or

#reviews_written = reviews.groupby('taster_twitter_handle').taster_twitter_handle.count()

taster_twitter_handle
@AnneInVino        3685
@JoeCz             5147
                   ... 
@winewchristina       6
@worldwineguys     1005
Length: 15, dtype: int64

## 2.
What is the best wine I can buy for a given amount of money? Create a `Series` whose index is wine prices and whose values is the maximum number of points a wine costing that much was given in a review. Sort the values by price, ascending (so that `4.0` dollars is at the top and `3300.0` dollars is at the bottom).

In [50]:
best_rating_per_price = reviews.groupby('price')['points'].max().sort_index()
best_rating_per_price


#choose prices with maximum points then sort them.
#Our group is base on price
#['points'].max() 




price
4.0       86
5.0       87
          ..
2500.0    96
3300.0    88
Name: points, Length: 390, dtype: int64

## 3.
What are the minimum and maximum prices for each `variety` of wine? Create a `DataFrame` whose index is the `variety` category from the dataset and whose values are the `min` and `max` values thereof.

In [52]:
price_extremes = reviews.groupby('variety').price.agg([min, max])
price_extremes 







,min,max
variety,,
Abouriou,15.0,75.0
Agiorgitiko,10.0,66.0
...,...,...
Çalkarası,19.0,19.0
Žilavka,15.0,15.0


## 4.
What are the most expensive wine varieties? Create a variable `sorted_varieties` containing a copy of the dataframe from the previous question where varieties are sorted in descending order based on minimum price, then on maximum price (to break ties).

In [53]:
sorted_varieties = price_extremes.sort_values(by=['min', 'max'], ascending=False)
sorted_varieties 







,min,max
variety,,
Ramisco,495.0,495.0
Terrantez,236.0,236.0
...,...,...
Vital,NaN,NaN
Zelen,NaN,NaN


## 5.
Create a `Series` whose index is reviewers and whose values is the average review score given out by that reviewer. Hint: you will need the `taster_name` and `points` columns.

In [55]:
reviewer_mean_ratings = reviews.groupby('taster_name').points.mean()
reviewer_mean_ratings







taster_name
Alexander Peartree    85.855422
Anna Lee C. Iijima    88.415629
                        ...    
Susan Kostrzewa       86.609217
Virginie Boone        89.213379
Name: points, Length: 19, dtype: float64

## 6.
What combination of countries and varieties are most common? Create a `Series` whose index is a `MultiIndex`of `{country, variety}` pairs. For example, a pinot noir produced in the US should map to `{"US", "Pinot Noir"}`. Sort the values in the `Series` in descending order based on wine count.

In [57]:
country_variety_counts = reviews.groupby(['country', 'variety']).size().sort_values(ascending=False)
country_variety_counts 







country    variety           
US         Pinot Noir            9885
           Cabernet Sauvignon    7315
                                 ... 
Italy      Moscato di Noto          1
Argentina  Barbera                  1
Length: 1612, dtype: int64